## Import functions and data

In [1]:
import nltk
from os import getcwd

In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

from utils import process_tweet, build_freqs

In [3]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Prepare the Data

In [4]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [5]:
# Split the data into train and test sets

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [6]:
# Combine positive and negative labels 
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

# Print out the shape of the test set
print("train_y.shape =", train_y.shape)
print("test_y.shape =", test_y.shape)

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [7]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) =", type(freqs))
print("len(freqs) =", len(freqs))

type(freqs) = <class 'dict'>
len(freqs) = 11346


In [8]:
# test process_tweet function
print("This is an example of a positive tweet: \n", train_x[0])
print("\nThis is an example of the process version of the tweet: \n", process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the process version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


# Part 1: Logistic Regression

### Part 1.1: Sigmoid

In [9]:
def sigmoid(z):
    """
    Input:
        z: the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    """
    h = 1 / (1 + np.exp(-z))
    return h

In [10]:
# Testing the function
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')
    
if (sigmoid(4.92) == 0.9927537604041685):
    print('Correct!')
else:
    print('Oops again!')

SUCCESS!
Correct!



### Part 1.2: Cost Function and Gradient

In [11]:
def gradient_descent(x, y, theta, alpha, num_iters):
    """
    Input:
        x: matrix of features which is (m, n+1)
        y: corresponding labels of matrix x, dimensions (m, 1)
        theta: weight vector of dimension (n+1, 1)
        alpha: learning rate
        num_iters: number of iterations you want to train the model for
    Output:
        J: the final cost
        theta: your final weight vector 
    """
    # get m, the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        # get the logit, dot product of x and theta
        z = np.dot(x, theta)
        # get the sigmoid of z
        h = sigmoid(z)
        # calculate the cost function
        J = (-1 / m) * ((np.dot(np.transpose(y), np.log(h)) + np.dot(np.transpose((1 - y)), np.log((1 - h)))))
        # update the weights theta
        theta = theta - (alpha / m) * np.dot(np.transpose(x), (h - y))
    
    J = float(J) 
    return J, theta

In [12]:
# Check the function

# dummy data
np.random.seed(1)
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
tmp_y = (np.random.rand(10, 1) > 0.35).astype(float)

# apply gradient descent
tmp_J, tmp_theta = gradient_descent(tmp_X, tmp_y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


## Extracting the features


In [13]:
def extract_features(tweet, freqs): 
    """
    Input:
        tweet: a list of words from one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1, 3)
    """
    # process_tweet tokenizes, stems, and removes stopwords
    word_1 = process_tweet(tweet)
    
    # 3 elements in the form of 1 x 3 vector 
    x = np.zeros((1, 3))
    
    # set bias to 1
    x[0][0] = 1
    
    for word in word_1:
        # increment the word count for positive label 1
        x[0][1] += freqs.get((word, 1), 0)
        # increment the word count for negative label 0
        x[0][2] += freqs.get((word, 0), 0)
        
    assert(x.shape == (1, 3))
    return x

In [14]:
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [15]:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


## Part 3: Training the Model

In [16]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i] = extract_features(train_x[i], freqs)

Y = train_y

J, theta = gradient_descent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216529
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


## Part 4: Test the Logistic Regression model

In [17]:
def predict_tweet(tweet, freqs, theta):
    """
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequency of each tuple (word, label)
        theta: (3, 1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    """
    # extract the features and store in x
    x = extract_features(tweet, freqs)
    
    # make predictions 
    z = np.dot(x, theta)
    y_pred = sigmoid(z)
    
    return y_pred

In [18]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print('%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561


### Check the performance using Test Set

In [19]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    # list for storing predictions
    y_hat = [1 if predict_tweet(tweet, freqs, theta) > 0.5 else 0 \
             for tweet in test_x]
    
    # convert the correct y's and predicted y's to 1D arrays and get the accuracy
    accuracy = sum(np.squeeze(y_hat) == np.squeeze(test_y)) / len(y_hat)
    
    return accuracy
    
        

In [20]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression's model accuracy = {tmp_accuracy:.4f}")

Logistic regression's model accuracy = 0.9950


In [21]:
# Some error analysis
print('Label Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = predict_tweet(x, freqs, theta)
    
    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS: ', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ''.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
THE PROCESSED TWEET IS:  ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']
1	0.49996890	b'trulilatermoveknowqueenbeeupwardboundmovingonup'
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS:  ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48622857	b'surewouldgoodthing4bottomdare2say2missbimgonnastubbornmouthsoapnothavingit:p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS:  ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48370665	b"i'mplaybraindotbraindot"
THE TWEET IS: I'm playing Brain Dots : ) #Brai